# Deep Dive into LangChain

## import the openAI key

In [3]:
from dotenv import load_dotenv
import os
load_dotenv("./.env")
API_KEY = os.getenv("API_OAI_KEY")

## ChatModels: GPT-3.5-Turbo and GPT-4

In [7]:
from langchain_openai import ChatOpenAI

prompt = "Explain Quantum Mechanics in one sentence"

llm = ChatOpenAI(api_key=API_KEY, model="gpt-4o-mini")
output = llm.invoke(prompt)
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the smallest scales, where particles exhibit wave-particle duality and uncertainty principles, leading to phenomena that defy classical intuition.


In [8]:
# Defining the roles using LangChain
from langchain.schema import (
	SystemMessage,
	AIMessage,
	HumanMessage

)

In [9]:
messages = [
	SystemMessage(content="Your are a physisit and respond only in German"),
	HumanMessage(content="Explain Quantum Mechanics in one sentence")
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik ist die Theorie, die das Verhalten von Materie und Energie auf der kleinsten Skala, der atomaren und subatomaren Ebene, beschreibt und dabei Konzepte wie Superposition und Verschränkung einführt.


## Caching LLM Responses

### In-memory cache

In [12]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name = "gpt-3.5-turbo-instruct", api_key=API_KEY)

In [13]:
%%time 
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = "Tell me a joke that a toddler can understand."

llm.invoke(prompt)

CPU times: user 55.4 ms, sys: 18.6 ms, total: 74 ms
Wall time: 1.06 s


"\n\nWhy did the banana go to the doctor?\nBecause it wasn't peeling well!"

In [14]:
%%time
llm.invoke(prompt)

CPU times: user 238 μs, sys: 45 μs, total: 283 μs
Wall time: 286 μs


"\n\nWhy did the banana go to the doctor?\nBecause it wasn't peeling well!"

## LLM Streaming

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=API_KEY)
prompt = "Write a rock song about the Moon and a Raven."
for chunk in llm.stream(prompt):
	print(chunk.content, end="", flush=True)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable